# Get all the vocabulary

This processes the dataset and gets the tokens, and assigns and ID to each one.

Make the vocabulary key
* ID 0 is _UNKNOWN_

In [ ]:
import os, sys, re
import gzip

In [ ]:
import gensim

In [ ]:
import numpy as np

In [ ]:
# Set the path to the downloaded GenSim word vectors file that you downloaded (see README)
model = gensim.models.KeyedVectors.load_word2vec_format('/media/thomas/026919b3-ea3e-4923-96aa-7f83aae1d652/pretrained_gensim/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Where the txt files are
input_folder = 'data/raw'

In [ ]:
output_folder = 'data/processed'

In [ ]:
model_folder = 'output'

In [ ]:
import pickle as pkl, gzip, pandas as pd
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|\'')

In [ ]:
def preprocess(text):
    tokens = tokenizer.tokenize(text)
    preprocessed  = []
    for token in tokens:
        if len(token) > 2 and token == token[0].upper() + token[1:].lower():
            token = "_PROPERNAME_"
        preprocessed.append(token.lower())
    return preprocessed

In [ ]:
all_vocab_used = {}

In [ ]:
for root, folder, files in os.walk(input_folder):
    for file_name in files:
        with open(input_folder + '/' + file_name, 'r') as f:
            all_lines = list(f)
            for l in all_lines:
                tokens = preprocess(l)
                for t in tokens:
                    if t in model.vocab:
                        if t not in all_vocab_used:
                            all_vocab_used[t] = 0
                        all_vocab_used[t] = all_vocab_used[t] + 1

In [ ]:
print ("Vocab size", len(all_vocab_used))

In [ ]:
import operator
sorted_vocab = list(sorted(all_vocab_used.items(), key=operator.itemgetter(1), reverse=True))

In [ ]:
sorted_vocab = [("_UNKNOWN_",-1)] + sorted_vocab[:9999]

In [ ]:
sorted_vocab[:10]

In [ ]:
abridged_index2word = []
abridged_word2index = {}
ctr = 0
for w, f in sorted_vocab:
    abridged_index2word.append(w)
    abridged_word2index[w] = ctr
    ctr += 1

In [ ]:
import gzip, pickle as pkl
with gzip.GzipFile("abridged_index2word.pkl.gz", "wb") as f:
    pkl.dump(abridged_index2word, f)
with gzip.GzipFile("abridged_word2index.pkl.gz", "wb") as f:
    pkl.dump(abridged_word2index, f)

In [ ]:
LENGTH_OF_GENSIM_VECTOR = 300

In [ ]:
# store the embeddings in a numpy array

embedding_matrix = np.zeros((len(abridged_index2word), LENGTH_OF_GENSIM_VECTOR))
for i in range(len(abridged_index2word)):
    if i == 0:
        continue #  _UNKNOWN_
    if i % 10000 == 0:
        print (i, "of", len(abridged_index2word))
    embedding_vector = model.wv[abridged_index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
#free memory
del(model)

In [ ]:
filter_sizes = "3,4,5"
num_filters = 128
l2_reg_lambda = 0
num_checkpoints = 10

In [ ]:
import tensorflow as tf

In [ ]:
import text_cnn
cnn = text_cnn.TextCNN(
            sequence_length=1000,
            num_classes=2, 
            vocab_size=(len(abridged_index2word)),
            embedding_size=LENGTH_OF_GENSIM_VECTOR,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)

In [ ]:
global_step = tf.Variable(0, name="global_step", trainable=False)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
tf.initialize_all_variables()

In [ ]:
embedding_placeholder = tf.placeholder(tf.float32, [len(abridged_index2word), LENGTH_OF_GENSIM_VECTOR])
embedding_init = cnn.W.assign(embedding_placeholder)

In [ ]:
tf.global_variables()

In [ ]:
gensim_weights = [x for x in tf.global_variables() if "embedding" in str(x)]
other_weights = [x for x in tf.global_variables() if "embedding" not in str(x)]

In [ ]:
gensim_weights

In [ ]:
sess.run(embedding_init, feed_dict={embedding_placeholder: embedding_matrix})

In [ ]:
saver_embedding = tf.train.Saver(gensim_weights)

In [ ]:
saver_embedding.save(sess, model_folder + "/gensim_weights")